<a href="https://colab.research.google.com/github/VictoryChianumba/yt-java-game/blob/master/Speech_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install libasound2-dev portaudio19-dev
libportaudio2 libportaudiocpp0 ffmpeg

In [ ]:
!pip install PyAudio

In [ ]:
%pip install torch

In [ ]:
%pip install vosk

In [ ]:
%pip install transformers

In [23]:
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
from queue import Queue

messages = Queue()
recordings = Queue()

record_button = widgets.Button(
    description = "Record",
    disable = False,
    button_style = "Success",
    icon = "microphone"
)

stop_button = widgets.Button(
    description = "Stop",
    disable = False,
    button_style = "warning",
    icon="stop"
)

output = widgets.Output()

def start_recording(data):
  messages.put(True)

  with output:
    display("Starting...")
    record = Thread(target = record_microphone)
    record.start()

    transcribe = Thread(target= speech_recognition, args=(output,))

def stop_recording(data):
  with output:
    messages.get()
    display("Stopped")

record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()

In [16]:
import pyaudio

p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
  print(p.get_device_info_by_index(i))

p.terminate()

In [17]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS= 20
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk = 1024):
  p = pyaudio.PyAudio()

  stream = p.open(format = AUDIO_FORMAT,
                  channels = CHANNELS,
                  rate = FRAME_RATE,
                  input = True,
                  input_device_index=2,
                  frames_per_buffer=chunk
                  )

  frames = []

  while not messages.empty():
    data = stream.read(chunk)
    frames.append(data)

    if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
      recordings.put(frames.copy())
      frames = []

  stream.stop_stream()
  stream.close()
  p.terminate()

In [21]:
import subprocess
import json
from vosk import Model, KaldiRecognizer

model = Model(model_name="vosk-model-small-en-us-0.15")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

def speech_recognition(output):
  while not messages.empty():
    frames = recordings.get()

    rec.AcceptWaveform(b''.join(frames))
    result = rec.Result()
    text = json.loads(result)["text"]

    #cased = subprocess.check_output()


vosk-model-small-en-us-0.15.zip: 100%|██████████| 39.3M/39.3M [00:03<00:00, 12.8MB/s]
